# ESiWACE3 Compression Lab in WASM

**Note:** Any changes you make to this notebook will be lost once the page is closed or refreshed. Please download any files you would like to keep.

**Note:** The WASM-based version of the compression lab running inside JupyterLite has only been tested in recent Chrome, Firefox, and Safari browsers.

## Patch HTTP requests for Pyodide kernels in JupyterLite

In [ ]:
import pyodide_http
pyodide_http.patch_all()

## Fetch an example dataset

### (a) Download a small dataset into memory

Small datasets can be downloaded and saved in the in-memory file system of this JupyterLite notebook. Note, however, that memory is limited.

In [ ]:
import urllib.request
import shutil

with urllib.request.urlopen(
    "https://juntyr.github.io/field-compression-lite/files/hplp_ml_q_dx=2.0.grib"
) as response:
    with open("hplp_ml_q_dx=2.0.grib", "wb") as file:
        shutil.copyfileobj(response, file)

### (b) Mount a large dataset from the user's filesystem

Alternatively, you can also upload a GRIB file from your own computer. The file is mounted in read-only mode into the notebook's file system without reading the file into memory, thus allowing arbitrarily large files to be made accessible. It is worth remembering that large files can still only be read if the algorithm that processes them supports streaming or chunking and does not request to load all data into memory at the same time.

If this notebook is run inside JupyterLite, the file also never leaves your own computer.

In [ ]:
%pip install ipyfilite
import ipyfilite

uploader = ipyfilite.FileUploadLite()
uploader

Once you have selected a file to upload, you can retrieve the path at which the file is now mounted in read-only mode.

In [ ]:
upload_path = uploader.value[0].path if len(uploader.value) > 0 else None
upload_path

### (c) Mount a large dataset from a URL that supports HTTP Range Requests

Similarly, you can also mount a file from a URL. The file is mounted in read-only mode into the notebook's file system without reading the file into memory, thus allowing arbitrarily large files to be made accessible. Note that the HTTP endpoint at the URL must support [range requests](https://developer.mozilla.org/en-US/docs/Web/HTTP/Range_requests) so that only small chunks of the file are loaded upon request. It is also worth remembering that large files can still only be read if the algorithm that processes them supports streaming or chunking and does not request to load all data into memory at the same time.

In [ ]:
%pip install ipyfilite
import ipyfilite

http_path = ipyfilite.HTTPFileIO(
    name="hplp_ml_q_dx=2.0.grib", url="https://juntyr.github.io/field-compression-lite/files/hplp_ml_q_dx=2.0.grib",
).path
http_path

## Load the example dataset

In [ ]:
%pip install ipywidgets
import fcpy

In [ ]:
ds = fcpy.open_dataset(
    # (a) only uncomment the following line to open the small example dataset
    "hplp_ml_q_dx=2.0.grib"
    # (b) only uncomment the following line to open your uploaded dataset
    # str(upload_path)
    # (c) only uncomment the following line to open the HTTP-mounted dataset
    # str(http_path)
)
ds = ds[["q"]]
ds

## Plot the dataset

In [ ]:
fcpy.suite.plot_spatial_single(
    ds["q"].sel(dict(hybrid=1)),
    ds,
    "q",
    metric="Baseline",
)

## Run a simple compressor

In [ ]:
compressed_linquant_2 = fcpy.suite.run_compressor_single(ds["q"], fcpy.compressors.LinQuantization(), bits=2)

In [ ]:
fcpy.suite.plot_spatial_single(
    compressed_linquant_2.sel(dict(hybrid=1)),
    ds,
    "q",
    metric="Compressed (bits=2)",
)

## Plot the simple compression error

In [ ]:
fcpy.suite.plot_spatial_single(
    compressed_linquant_2.sel(dict(hybrid=1)) - ds["q"].sel(dict(hybrid=1)),
    ds,
    "q",
    metric="Compression Error (bits=2)",
)

## Run a better compressor

In [ ]:
compressed_linquant_24 = fcpy.suite.run_compressor_single(ds["q"], fcpy.compressors.LinQuantization(), bits=24)

In [ ]:
fcpy.suite.plot_spatial_single(
    compressed_linquant_24.sel(dict(hybrid=1)),
    ds,
    "q",
    metric="Compressed (bits=24)",
)

## Plot the better compression error

In [ ]:
fcpy.suite.plot_spatial_single(
    compressed_linquant_24.sel(dict(hybrid=1)) - ds["q"].sel(dict(hybrid=1)),
    ds,
    "q",
    metric="Compression Error (bits=24)",
)